In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score, KFold
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
# First Analysis

In [ ]:
HRisk_data = pd.read_csv("Maternal Health Risk Data Set.csv")
HRisk_data

In [ ]:
#### The target column is RiskLevel with three classes named: 'high risk', 'low risk', 'mid risk'

In [ ]:
HRisk_data["RiskLevel"].unique()

In [ ]:
# Data pre-processing

In [ ]:
### Removing NaN values

In [ ]:
HRisk_data = HRisk_data.dropna()
HRisk_data

In [ ]:
#### Converting mid risk and high risk to 1 and low risk to 0

In [ ]:
HRisk_data["RiskLevel"] = HRisk_data["RiskLevel"].replace({"low risk": 0, "mid risk": 1, "high risk": 1})
HRisk_data["RiskLevel"].unique()

In [ ]:
# plt.figure(figsize=(12, 10))
# sns.heatmap(HRisk_data.corr(), annot=True, cmap='coolwarm', fmt=".2f",
#                       annot_kws={"fontsize": 7})  # Adjust fontsize as needed
# plt.title("correlation heatmap")
plt.figure(figsize=(12, 10))
heatmap = sns.heatmap(HRisk_data.corr(), annot=True, cmap='coolwarm', fmt=".2f",
            annot_kws={"fontsize": 16})  # Adjust fontsize as needed
heatmap.set_xticklabels(heatmap.get_xticklabels(), fontsize=13)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=13)
plt.title("Correlation Heatmap", fontsize=16)  # Adjust fontsize as needed
plt.show()

In [ ]:
lowRisk_count = HRisk_data[HRisk_data["RiskLevel"] == 0].RiskLevel.count()
Risky_count = HRisk_data[HRisk_data["RiskLevel"] == 1].RiskLevel.count()

plt.bar(['Low risk', 'Risky'], [lowRisk_count, Risky_count])
plt.xticks(fontsize=15)

for i, count in enumerate([lowRisk_count, Risky_count]):
    plt.text(i, count + 0.1, str(count), ha='center', fontsize=15)

plt.xlabel("Target names", fontsize=15)
plt.ylabel("Counts", fontsize=15)
plt.title("Counts of each target names", fontsize=15)

In [ ]:
data = HRisk_data[["SystolicBP", "DiastolicBP", "BS", "RiskLevel", "BodyTemp"]]
data

In [ ]:
X = HRisk_data.drop(["RiskLevel", "HeartRate"], axis=1).values.copy()
y = HRisk_data["RiskLevel"].values.copy()


In [ ]:
### Defining the decision boundaries

In [ ]:
y_data = y
X_data_Sys_Dia = HRisk_data[["SystolicBP", "DiastolicBP"]]
X_data_Sys_BS = HRisk_data[["SystolicBP", "BS"]]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC

fig, axs = plt.subplots(1, 2, figsize=(17, 5))




clf = SVC(kernel='linear')
# clf = SVC(kernel='linear', class_weight={0: 1, 1: 1})
clf.fit(X_data_Sys_Dia, y_data)
x_min, x_max = X_data_Sys_Dia.iloc[:, 0].min() - 1, X_data_Sys_Dia.iloc[:, 0].max() + 1
y_min, y_max = X_data_Sys_Dia.iloc[:, 1].min() - 1, X_data_Sys_Dia.iloc[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
axs[0].contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
# Plot class samples
axs[0].scatter(X_data_Sys_Dia.iloc[:, 0], X_data_Sys_Dia.iloc[:, 1], c=y_data, cmap='coolwarm', edgecolors='k')
plt.xticks(fontsize=14)
axs[0].set_xlabel('SystolicBP', fontsize = 14)
axs[0].set_ylabel('DiastolicBP', fontsize = 14)
axs[0].set_title("SVM with kernel linear decision boundary for SystolicBP and DiastolicBP", fontsize = 14)
axs[0].tick_params(axis='x', labelsize=12)
axs[0].tick_params(axis='y', labelsize=12)



color = ['b' if y == 0 else "r" for y in HRisk_data["RiskLevel"]]
lm = LogisticRegression()
lm.fit(HRisk_data[["SystolicBP", "BS"]], HRisk_data["RiskLevel"])
axs[1].scatter(HRisk_data["SystolicBP"], HRisk_data["BS"], c=color)
b0 = lm.intercept_[0]
b = lm.coef_[0]
x_ = [50, 150]
y_ = [-(b0 + b[0] * x)/b[1] for x in x_]
axs[1].plot(x_, y_)
axs[1].set_xlabel("SystolicBP", fontsize = 14)
axs[1].set_ylabel("BS", fontsize = 14)
axs[1].set_title("Logistic regression decision boundary for BS and SystolicBP", fontsize = 14)
axs[1].tick_params(axis='x', labelsize=12)
axs[1].tick_params(axis='y', labelsize=12)
display(f"b0 -> {b0}")
display(f"b1 -> {b[0]}")
display(f"b2 -> {b[1]}")



### SVM kernel with systolic and diastolic

In [ ]:
X_train_SysDias, X_test_SysDias, y_train_SysDias, y_test_SysDias = train_test_split(HRisk_data[["SystolicBP", "DiastolicBP"]], y_data, test_size=0.2, random_state=42)
clf = SVC(kernel='linear')
clf.fit(X_train_SysDias, y_train_SysDias)
predicted = clf.predict(X_test_SysDias)
p,r,f,s = precision_recall_fscore_support(y_test_SysDias, predicted, labels=[0, 1])
display('precision = {}'.format(p))
display('recall = {}'.format(r))
display('f-score = {}'.format(f))

### Logistic regression on `SystolicBP`, `BS`

In [ ]:
X_train_SysDias, X_test_SysDias, y_train_SysDias, y_test_SysDias = train_test_split(HRisk_data[["SystolicBP", "BS"]], y_data, test_size=0.2, random_state=42)
clf = LogisticRegression()
clf.fit(X_train_SysDias, y_train_SysDias)
predicted = clf.predict(X_test_SysDias)
p,r,f,s = precision_recall_fscore_support(y_test_SysDias, predicted, labels=[0, 1])
display('precision = {}'.format(p))
display('recall = {}'.format(r))
display('f-score = {}'.format(f))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Logistic Regression

In [ ]:
LRmodel = LogisticRegression(max_iter=1000)
LRmodel.fit(X_train, y_train)
y_pred_LR = LRmodel.predict(X_test)



In [ ]:
p,r,f,s = precision_recall_fscore_support(y_test, y_pred_LR, labels=[0, 1])

display('precision = {}'.format(p))
display('recall = {}'.format(r))
display('f-score = {}'.format(f))

In [ ]:
# SVM linear

In [ ]:
model = SVC(kernel="linear")
model.fit(X_train, y_train)
y_pred_LSVM = model.predict(X_test)



In [ ]:
p,r,f,s = precision_recall_fscore_support(y_test, y_pred_LSVM, labels=[0, 1])

display('precision = {}'.format(p))
display('recall = {}'.format(r))
display('f-score = {}'.format(f))

In [ ]:
# SVM linear changing the class weights

In [ ]:
model = SVC(kernel="linear", class_weight={0: .9, 1: 1})
model.fit(X_train, y_train)
y_pred_LSVM_ClassWeights = model.predict(X_test)


In [ ]:
p,r,f,s = precision_recall_fscore_support(y_test, y_pred_LSVM_ClassWeights, labels=[0, 1])

display('precision = {}'.format(p))
display('recall = {}'.format(r))
display('f-score = {}'.format(f))

In [ ]:
model = SVC(kernel="poly")
model.fit(X_train, y_train)
y_pred_SVM_poly = model.predict(X_test)



In [ ]:
p,r,f,s = precision_recall_fscore_support(y_test, y_pred_SVM_poly, labels=[0, 1])
display('precision = {}'.format(p))
display('recall = {}'.format(r))
display('f-score = {}'.format(f))

In [ ]:
# SVM polynomial changing class weight

In [ ]:
model = SVC(kernel="poly", class_weight={0: .9, 1: 1})
model.fit(X_train, y_train)
y_pred_poly_classWeights = model.predict(X_test)


In [ ]:
p,r,f,s = precision_recall_fscore_support(y_test, y_pred_poly_classWeights, labels=[0, 1])

display('precision = {}'.format(p))
display('recall = {}'.format(r))
display('f-score = {}'.format(f))

In [ ]:
# SVM RBF


In [ ]:
model = SVC(kernel="rbf")
model.fit(X_train, y_train)
y_pred_RBF = model.predict(X_test)


In [ ]:
p,r,f,s = precision_recall_fscore_support(y_test, y_pred_RBF, labels=[0, 1])

display('precision = {}'.format(p))
display('recall = {}'.format(r))
display('f-score = {}'.format(f))

In [ ]:
# SVM RBF with changing class weights

In [ ]:
model = SVC(kernel="rbf", class_weight={0: .9, 1: 1})
model.fit(X_train, y_train)
y_pred_RBF_ClassWeights = model.predict(X_test)


In [ ]:
p,r,f,s = precision_recall_fscore_support(y_test, y_pred_RBF_ClassWeights, labels=[0, 1])

display('precision = {}'.format(p))
display('recall = {}'.format(r))
display('f-score = {}'.format(f))

In [ ]:
# Second analysis

In [ ]:
data = pd.read_csv("apple_quality.csv")
data


In [ ]:
data = data.dropna()
data = data.drop('A_id',axis=1)
data['Acidity'] = data['Acidity'].astype(float)
data['Quality'] = data['Quality'].replace({'good': 1, 'bad': 0})

##data = data.head(1000)

data


In [ ]:
# plt.figure(figsize=(12, 10))
# sns.heatmap(HRisk_data.corr(), annot=True, cmap='coolwarm', fmt=".2f",
#                       annot_kws={"fontsize": 7})  # Adjust fontsize as needed
# plt.title("correlation heatmap")
plt.figure(figsize=(17, 14))
heatmap = sns.heatmap(data.corr(), annot=True, cmap='coolwarm', fmt=".2f",
            annot_kws={"fontsize": 23})  # Adjust fontsize as needed
heatmap.set_xticklabels(heatmap.get_xticklabels(), fontsize=17)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=17)
plt.title("Correlation Heatmap", fontsize=25)  # Adjust fontsize as needed
plt.show()


In [ ]:
from sklearn.linear_model import LogisticRegression

# X = data[['Size','Sweetness']]
X = data[['Ripeness','Juiciness']]
y = data.Quality

lm = LogisticRegression()
lm.fit(X, y)

# print(lm.intercept_)
# print(lm.coef_)

y_pred = lm.predict(X)

display(y_pred)

display('predicted range: [{0:.2f},{1:.2f}]'.format(min(y_pred), max(y_pred)))

p,r,f,s = precision_recall_fscore_support(y, y_pred, labels=[0, 1])
display('precision = {}'.format(p))
display('recall = {}'.format(r))
display('f-score = {}'.format(f))


In [ ]:
color = ['r' if y_ == 0 else 'b' for y_ in y]
plt.scatter(data.Juiciness, data.Ripeness, c=color, s=8)

b0 = lm.intercept_[0]
b = lm.coef_[0]

print(b0, b)
# Xdb = [-4.8, 2.8]
Xdb = [-2, 6]
ydb = [-(b0+b[0]*x)/b[1] for x in Xdb]
plt.plot(Xdb, ydb)
plt.ylabel('Ripeness')
plt.xlabel('Juiciness')
plt.title("Ripeness - Juiciness Logistic regression decision boundary")


In [ ]:
df = pd.read_csv("apple_quality.csv")


In [ ]:
df = df.dropna()
df = df.drop('A_id',axis=1)
df['Acidity'] = df['Acidity'].astype(float)
df['Quality'] = df['Quality'].replace({'good': 1, 'bad': 0})


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import f1_score, recall_score
from sklearn.preprocessing import StandardScaler


X = df.drop("Quality", axis=1).values.copy()
y = df["Quality"].values.copy()

display(X)


# Step 4: Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train SVM Model
# Optional: Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)






In [ ]:
# Create SVM model
svm_model = SVC(kernel='linear')  # You can choose different kernels like 'rbf', 'poly', etc.

# Train the model
svm_model.fit(X_train_scaled, y_train)

# Step 6: Evaluate Model
# Make predictions
y_pred = svm_model.predict(X_test_scaled)

In [ ]:

# # Calculate F1-score
# f1 = f1_score(y_test, y_pred)
# print("F1-score:", f1)

# # Calculate recall
# recall = recall_score(y_test, y_pred)
# print("Recall:", recall)

p,r,f,s = precision_recall_fscore_support(y_test, y_pred, labels=[0, 1])
display('precision = {}'.format(p))
display('recall = {}'.format(r))
display('f-score = {}'.format(f))

In [ ]:

cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(4, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix of SVM (linear)')
plt.show()
